# CNN model

In [1]:
import deeplake
from skimage.color import rgb2gray
import numpy as np
from PIL import Image
from sklearn import svm
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn import metrics
import sklearn
import time
import sys
from IPython.display import clear_output
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import wandb
from wandb.keras import WandbCallback
import matplotlib.pyplot as plt
from datetime import datetime
from collections import Counter
from itertools import chain
clear_output()

In [2]:
date_and_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [3]:
val_ds = deeplake.load('hub://luizapzbn/CODEBRIM_classification_dataset-val')
train_ds = deeplake.load('hub://luizapzbn/CODEBRIM_classification_dataset-train')

hub://luizapzbn/CODEBRIM_classification_dataset-val loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/luizapzbn/CODEBRIM_classification_dataset-val
hub://luizapzbn/CODEBRIM_classification_dataset-train loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/luizapzbn/CODEBRIM_classification_dataset-train


# Loading in the datasets and normalizing them

In [4]:
# train set
X_train = (train_ds.images.numpy()/255).astype('float32')
y_train = train_ds.labels.data()
train_labels = list(chain.from_iterable(train_ds.labels.data()['text']))

# validation data
X_val = (val_ds.images.numpy()/255).astype('float32')
y_val = val_ds.labels.data()
val_labels = list(chain.from_iterable(val_ds.labels.data()['text']))

# Label values and label texts
y_train_vals = y_train['value']
y_train_text = y_train['text']
y_val_vals = y_val['value']
y_val_text = y_val['text']

## Model for RGB pictures - 6 classes

In [5]:
def create_cnn(input_shape, output_classes):
    return keras.Sequential(
        [                                                       
            keras.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(256, 256, 3)),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu", input_shape=(256, 256, 3)),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dense(output_classes, activation="softmax")
        ]
    )

In [7]:
# # convert class vectors to binary class matrices
num_classes = 6
y_train_vals = keras.utils.to_categorical(y_train_vals, num_classes)
y_val_vals = keras.utils.to_categorical(y_val_vals, num_classes)

In [9]:
batch_size = 0
epochs = 5

In [18]:
wandb.init(project="inlamning1V1", name=f"preprocessing {date_and_time}", config = {
    "training_type": 'Convolutional Neural Network',
    "learning_rate": 0.001,
    "epochs": epochs,
    "batch_size": batch_size
    })


# Create a Convolutional Neural Network that
model = create_cnn((256, 256, 3), 6)

model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=["accuracy"])
model.fit(X_train, y_train_vals, batch_size=batch_size,
          epochs=epochs, validation_split=0.1, callbacks=[wandb.keras.WandbCallback()])

In [ ]:
class_names = ['Background', 'Efflorescence', 'Crack', 'Spallation', 'CorrosionStain', 'ExposedBars']

# Precision recall curve
wandb.log({"pr": wandb.plot.pr_curve(val_labels, predictions_cnn)})
# ROC curve
wandb.log({"roc": wandb.plot.roc_curve(val_labels, predictions_cnn)})
# Confusion matrix
cm = wandb.plot.confusion_matrix(
    y_true=y_val_vals,
    preds=predictions_cnn,
    class_names=class_names)
    
wandb.log({"conf_mat": cm})

wandb.finish()

## CNN - Grayscale images

In [11]:
# greying the pictures
X_train=rgb2gray(X_train)
X_val=rgb2gray(X_val)

In [12]:
y_train_vals_extended = np.eye(6)[y_train_vals.reshape(-1)].reshape(6481, 6)
y_train_vals_extended.shape

(6481, 6)

In [13]:
# # Make sure images have shape (256, 256, 1)
X_train = np.expand_dims(X_train, -1)
X_val = np.expand_dims(X_val, -1)

## Model for grayscale images - 6 classes 

In [8]:
def create_cnn(input_shape, output_classes):
    return keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(256, 256, 1)),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu", input_shape=(256, 256, 1)),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dense(output_classes, activation="softmax")
        ]
    )

In [9]:
batch_size = 0
epochs = 5

In [20]:
wandb.init(project="inlamning1TEST", name=f"preprocessing {date_and_time}", config = {
    "training_type": 'Convolutional Neural Network',
    "learning_rate": 0.001,
    "epochs": epochs,
    "batch_size": batch_size
    })


# Create a Convolutional Neural Network that
model = create_cnn((256, 256, 1), 6)

model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=["accuracy"])
model.fit(X_train, y_train_vals_extended, batch_size=batch_size,
          epochs=epochs, validation_split=0.1, callbacks=[wandb.keras.WandbCallback()])

## Predictions

In [11]:
predictions_cnn = model.predict(X_val)
# prediction to list
pred_list = [np.argmax(x) for x in predictions_cnn]
# y_val_vals to list
y_val_vals_list = list(chain.from_iterable(y_val_vals))

20/20 [==============================] - 0s 10ms/step


# Evaluation

In [ ]:
class_names = ['Background', 'Efflorescence', 'Crack', 'Spallation', 'CorrosionStain', 'ExposedBars']

# Precision recall curve
wandb.log({"pr": wandb.plot.pr_curve(val_labels, predictions_cnn)})
# ROC curve
wandb.log({"roc": wandb.plot.roc_curve(val_labels, predictions_cnn)})
# Confusion matrix
cm = wandb.plot.confusion_matrix(
    y_true=y_val_vals_list,
    preds=pred_list,
    class_names=class_names)
    
wandb.log({"conf_mat": cm})

wandb.finish()

## Model for grayscale images - 2 classes

In [14]:
y_train_binary_value = np.where(y_train_vals!=[0], [1], [0])
print('Binary values: ', Counter(list(chain.from_iterable(y_train_binary_value))))

y_train_binary_text = np.where(np.array(y_train_text)!=['Background'], ['Damaged'], ['Not damaged'])
print('Binary texts: ', Counter(list(chain.from_iterable(y_train_binary_text))))

y_val_binary_value = np.where(y_val_vals!=[0], [1], [0])
print('Binary values: ', Counter(list(chain.from_iterable(y_val_binary_value))))

y_val_binary_text = np.where(np.array(y_val_text)!=['Background'], ['Damaged'], ['Not damaged'])
print('Binary texts: ', Counter(list(chain.from_iterable(y_val_binary_text))))

Binary values:  Counter({1: 4253, 0: 2228})
Binary texts:  Counter({'Damaged': 4253, 'Not damaged': 2228})
Binary values:  Counter({1: 461, 0: 155})
Binary texts:  Counter({'Damaged': 461, 'Not damaged': 155})


In [15]:
def create_cnn(input_shape, output_classes):
    return keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(256, 256, 1)),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu", input_shape=(256, 256, 1)),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dense(output_classes, activation="softmax")
        ]
    )

In [16]:
# # convert class vectors to binary class matrices
num_classes = 2
y_train_binary_value = keras.utils.to_categorical(y_train_binary_value, num_classes)
y_val_binary_value = keras.utils.to_categorical(y_val_binary_value, num_classes)
# y_val_text = keras.utils.to_categorical(y_val_text, num_classes)

In [17]:
batch_size = 0
epochs = 5
date_and_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [19]:
wandb.init(project="inlamning1TEST", name=f"preprocessing {date_and_time}", config = {
    "training_type": 'Convolutional Neural Network',
    "learning_rate": 0.001,
    "epochs": epochs,
    "batch_size": batch_size
    })


# Create a Convolutional Neural Network that
model = create_cnn((256, 256, 1), 2)

model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=["accuracy"])
model.fit(X_train, y_train_binary_value, batch_size=batch_size,
          epochs=epochs, validation_split=0.1, callbacks=[wandb.keras.WandbCallback()])

## Predictions

In [ ]:
predictions_cnn = model.predict(X_val)
# prediction to list
pred_list = [np.argmax(x) for x in predictions_cnn]
# y_val_vals to list
y_val_vals_list = list(chain.from_iterable(y_val_vals))

# Evaluation

In [ ]:
class_names = ['Background', 'Efflorescence', 'Crack', 'Spallation', 'CorrosionStain', 'ExposedBars']

# Precision recall curve
wandb.log({"pr": wandb.plot.pr_curve(val_labels, predictions_cnn)})
# ROC curve
wandb.log({"roc": wandb.plot.roc_curve(val_labels, predictions_cnn)})
# Confusion matrix
cm = wandb.plot.confusion_matrix(
    y_true=y_val_vals_list,
    preds=pred_list,
    class_names=class_names)
    
wandb.log({"conf_mat": cm})

wandb.finish()